### Static Approach: Modeling the relative glycan distribution in monoclonal antibody using deep learning 

The development of a model that uses deep learning to predict the final relative glycan distribution of an antibody has been discussed here. The framework also known as the "Static approach" uses data driven techniques instead of first priciples based methods to predict glycan distribution. The aim of this project is not to understand the underlying physics behind glycosylation in monoclonal antibodies but to develop an efficient model that provides accurate and instantaneous predictions of the glycan distribution from experimental data. The model is also independent of cell lines and the addition of supplements. 

The model building approach is divided into the following parts:

1. [Data Collection](#tag_1)
2. [Data Pre-processing](#tag_2)
3. [Data Visualization and Analysis](#tag_3)
4. [Deep Learning Model](#tag_4)
5. [Model Prediction](#tag_5)
6. [Model Evaluation](#tag_6)

## <a id="tag_1">Data Collection</a>  
We ran a series of experiments to collect the data required to build the model. 

### Experimental Setup:

An autoclavable bioreactor system provided by Applikon was used to conduct the experiments and gather data. The system consists for the following parts:
1. A **bio-reactor** with the appropriate auxiliaries like stirrer assembly, sensors, an aeration assembly etc. for process control.
2. A **my-Control bio-controller** for measurement and control of process variables (like pH, temperature, dO2, level and stirrer speed) with corresponding actuators in order to keep process conditions at set-point
3. A **host** PC that is used as a Human Machine Interface(HMI)
4. An **Open Platform Communication(OPC) server** that converts the hardware communication protocol into the OPC protocol. 
5. An **OPC client** that connects to the hardware. The OPC client uses the OPC server to get data from or send commands to the hardware. 

The OPC server software was provided by Applikon and it was pre-installed in my-Control bio-controller. We used MATLAB's OPC toolbox (OPC client) to interact with Applikon's OPC server for continuous data logging and sending acutuator commands to the bio-reactor. 

A schematic of the experimental setup along with the flow of information is given below:

![](setup_exp.png)

#### Bioreactor

*to be written*

#### my-Control

The my-Control is a biocontroller that is dedicated to the process in the bioreactor. It contains:
1. Fixed and adaptive PID control for pH, dO2 and Temperature
2. Stirrer Speed control
3. Foam and/or Level control
4. Additional (freely configurable) measuring channels / PID control loops, such as: (max. 4) Analog Inputs for sensors / measuring devices (max. 4) Scales (USB) for Feed Control Optical Sensors (USB) for pH and dO2 Biomass Monitor (USB)

Inside the my-Control, the following actuators are supported:
1. Variable speed tubing pumps (max. four mounting positions)
2. Gas Solenoid Valves with Tuning Valve for aeration (max. four internal mounting positions)
3. Three channel Mass Flow Controller for aeration (internal mounting)
4. Power Supply for heating (Heating Blanket or Thermo-Electric element) and cooling (Thermo-Electric element or connection of external cooling valve)

#### Host

Through a network that is connected to my-Control (TCP/IP communication), different kind of devices can be used as the Human Machine Interface. In our case, we used a PC as the HMI by establishing a LAN connection with my-Control using Ethernet. 

#### OPC server

OPC is a software interface standard that allows Windows programs to communicate with industrial hardware devices. OPC is implemented in server/client pairs. The OPC server is a software program that converts the hardware communication protocol used by a PLC into the OPC protocol.
We used Applikon's OPC server specially designed to control Applikon bioreactors by MatrikonOPC. 

#### OPC client

The OPC client software is any program that needs to connect to the hardware. The OPC client uses the OPC server to get data from or send commands to the hardware. We used MATLAB's OPC Toolbox client to connect to Applikon's OPC server. With the help of Simulink in MATLAB, we were able to continuously read and write data to and from the server.

#### Information Flow

The bioreactor has multiple sensors including temperature, pH, dissovled oxygen, stirrer speed sensors that transmit the current value of these variables to my-Control bio-controller. The bio-controller receives user-defined setpoints for these variables through it's OPC server. It has it's own PID controller that compares the setpoints with the current values provides commands to the actuators. The actuator acts accordingly and pushes the current value of the variables towards it's desired setpoint. The values of the variables throughout the timespan of the experiment are logged in host memory thorugh opc.  

### Experiment

*tobewritten*

### Experimental Data

The following information obtained from the experiment will be used to predict the glycan distribution. 

1. Glucose concentration profile
2. Glutamine concentration profile
3. Lactate concentration profile
4. Ammonia concentration profile
5. Viable cell density profile
6. Total cell density profile
7. Antibody concentration profile
8. The final relative glycan distribution

The next step involves pre-processing the data to prepare it for analysis and modeling.

## <a id="tag_2">Data Pre-processing</a>

### Discretizing the data and selecting timesteps 

The data obtained from the bio-reactor has been discretized into 4 timesteps (Day 0, Day 2, Day 4 and Day 7) covering the entire span of the experiment. This was done to decrease computational time and increase the efficiency of the model. For eg. the first five experimental data of glucose is as follows:

![](GlucoseTable.png)

Similarly, we have discretized Glutamine, Lactate, Ammonia Viable cell density, total cell density and Antibody concentration profile. Therefore our independent variables are as follows:

1. Glucose concentration at 4 timesteps
2. Glutamine concentration at 4 timesteps 
3. Lactate concentration at 4 timesteps
4. Ammonia concentration at 4 timesteps
5. Viable Cell Density at 4 timesteps
6. Total Cell Density at 4 timesteps
7. Antibody concentration at 4 timesteps 

### Normalizing the data

Since the range of values vary widely, the raw data for each of the independent variable was normalized using Z-score normalization. This step is important because we used mean squared error as our objective function which does not work properly without normalization. 

The final relative glycan distribution is already in normalized form. It is our dependent/output variable. 

### Creating feature space

Finally we combined all the independent variables into one feature space that has 28 columns (7 independent variables each having 4 timesteps) and perform statistical analysis on the data.

### The output

Our output variable was the relative glycan distribution of the antibody which determines the antibody's final product quality. *Write more ...* 

## <a id="tag_3">Data Visualization and Analysis</a>

The feature space created above is very high dimnesional and highly correlated. Theoretically, they are all time series data and each time step is dependent on the previous time steps. Moreover, antibody concentration is dependent on all of the nutrients (glucose, glutamine, lactate, ammonia) and the total and viable cell density. Mathematically, we looked at both pearson and spearman correlation to figure out the correlations among them. 

### Pearson's Correlation Result

Pearson's correlation captures any linear relationship between two variables. The heatmap generated from the analysis is given below:

![](pearson.png)

From the heatmap, it was evident that many variables are either positively or negatively correlated with each other. To be more specific, we found that among 756 interactions (28\*28 -28), 182 showed linear relation.  

### Spearman's Correlation Result

Spearman's correlation captures any motonic relation between two variables. The heatmap generated from the analysis is given below:

![](spearman.png)

In this case, we found that there were 118 monotonic relations. 

### Reduction of the feature space using PCA

Since we have enough evidence to suggest correlation between the features, we decided to use Principal Component Analysis to reduce the feature space. Principal Component Analysis is a statistical procedure that is used to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables. This transformation also helps us to select the principal components that maximize the variance in the data. 

From Pricipal Component Analysis, we found that 95% of the variance in the data is explained by the first 6 principal components and we will only use these 6 components in our model. Thus we have reduced the feature space from 28 to 6. This will further reduce the computation time of our model.

### Hypotheses Testing to govern the difference between Ambic and CHO cell line

*to be written*

## <a id="tag_4">Deep Learning Model</a>

A deep neural network model is trained on the reduced feature space to predict the final relative glycan distribution of the antibody. 

### Model Selection

Multiple iterations were carried out to select the best possible set of hyperparameters using scikit learn. 100 different neural networks were trained on each set of hidden layers and hidden units by varying the random state parameter of the scikit learn MLP Regressor class. Then we took the average of those 100 values to determine the best possible hidden layer and hidden unit combination. The result of the iterationsis given below.  

![](HyperParams.png)

It is clear from the above heatmap that the best possible combination of hyper parameters is 3 hidden unit and 4 hidden layers. 

### Model Architecture

The deep learning model architecture is as follows:

1. Hidden Units: 3
2. Hidden Layers: 4
3. Activation function: "logistic"
4. Learning rate: 0.001
5. Regularization penalty: L2
6. Regularization parameter: 0.0001
7. Number of iterations: 300
8. Tolerance: $1e^{-5}$

## <a id="tag_5">Model Prediction</a>

The model predictions on 3 test cases are shown below:

![](GTest1.png)
![](GTest2.png)
![](GTest3.png)

## <a id="tag_6">Model Evaluation</a>

The model score against various evaluation metrics is given below.

| Model Metric | Model Score |
| --- | --- |
| $r^{2}$ | 0.91031 |
| **MSE** | 0.00014 |
| **MAE** | 0.00899 |

Compared to this, a multivariate linear regression model gives the following scores on the test cases. 

| Model Metric | Model Score |
| --- | --- |
| $r^{2}$ | 0.78099 |
| **MSE** | 0.00035 |
| **MAE** | 0.01271 |